In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

/Users/isaiah/Developer/music-genre-classifier/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
DATA_PATH = "data/genres_original/"
GENRES = os.listdir(DATA_PATH)
IMG_SIZE = (128, 128)  # Resizing spectrograms

X, y = [], []

for genre_index, genre in enumerate(GENRES):
    genre_path = os.path.join(DATA_PATH, genre)
    
    for file in os.listdir(genre_path):
        file_path = os.path.join(genre_path, file)
        
        y_audio, sr = librosa.load(file_path, sr=22050)
        mel_spec = librosa.feature.melspectrogram(y=y_audio, sr=sr)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB scale
        
        # Resize to uniform shape
        mel_spec_resized = np.resize(mel_spec_db, IMG_SIZE)
        
        X.append(mel_spec_resized)
        y.append(genre_index)  # Store genre label as integer

X = np.array(X).reshape(-1, IMG_SIZE[0], IMG_SIZE[1], 1)  # Reshape for CNN
y = to_categorical(y, num_classes=len(GENRES))  # One-hot encode labels

/var/folders/3z/t4gl8nqj2rd9f933dwn34n4r0000gn/T/ipykernel_81643/488102470.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  y_audio, sr = librosa.load(file_path, sr=22050)


NoBackendError: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(GENRES), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 CNN Model Accuracy: {test_acc * 100:.2f}%")

from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes, target_names=GENRES))